# Experiment 05-01

In [170]:
import numpy as np
import hashlib
from binascii import hexlify
import pyopencl as cl
from Library.opencl_information import opencl_information

## Show the available Platforms

In [171]:
info = opencl_information()
info.print_full_info()


OpenCL Platforms and Devices
Platform 0 - Name: Apple
Platform 0 - Vendor: Apple
Platform 0 - Version: OpenCL 1.2 (Apr 19 2022 18:44:44)
Platform 0 - Profile: FULL_PROFILE
 --------------------------------------------------------
 Device - Name: Apple M1
 Device - Type: ALL | GPU
 Device - Max Clock Speed: 1000 Mhz
 Device - Compute Units: 8
 Device - Local Memory: 32 KB
 Device - Constant Memory: 1048576 KB
 Device - Global Memory: 5 GB
 Device - Max Buffer/Image Size: 1024 MB
 Device - Max Work Group Size: 256




## Configure the OpenCL Context

In [172]:
platform_number = 0
device_number = 0

cl_devices = cl.get_platforms()[platform_number].get_devices()
cl_ctx = cl.Context(cl_devices)
cl_queue = cl.CommandQueue(cl_ctx, cl_devices[device_number])

## Compile the Program

In [173]:
def build_program(program_files : list, cl_ctx : cl.Context,
        build_options=[]) -> cl.Program:
    """
    Build a program from an OpenCL source file.

    Parameters
    ----------
    program_files : list
        The path to the OpenCL source files.
    cl_ctx : pyopencl.Context
        The context to build the program with.
    build_options : list of str
        The build options to use.

    Returns
    -------
    pyopencl.Program
    """
    program_source = ''

    for cl_file in program_files:
        with open(cl_file, 'r') as cl_file:
            file_source = cl_file.read()
            program_source += '\n' + file_source

    program_source = cl.Program(cl_ctx, program_source)
    program = program_source.build(options=build_options)
            
    return program

In [174]:
cl_program_files = [
    'Library/worker/sha256.cl',
    'Library/worker/zimcoin.cl',
]

cl_program = build_program(cl_program_files, cl_ctx)

# show the kernel names
program_kernel_names = cl_program.get_info(cl.program_info.KERNEL_NAMES)
print(f"Kernel Names: {program_kernel_names}")

Kernel Names: hash_main;get_single_hash;get_random_numbers;get_random_string;get_single_hash_nonce


## Initial Tests

### Single Hash

In [175]:
# the plaintext to hash
#plaintext = 'bells hel the quick brown fox jumps over the lazy dog 12356'
plaintext = 'this is a description of the latest block' + '395707976'
plaintext_bytes = np.frombuffer(plaintext.encode('utf-8'), dtype=np.uint8)
plaintext_length = np.int32(len(plaintext_bytes))

# the hash output
hash_output = np.zeros(8, dtype=np.uint32)

# allocate the memory for the variables on the device
cl_plaintext_bytes = cl.Buffer(
    cl_ctx,
    cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR,
    hostbuf=plaintext_bytes)

cl_plaintext_length = cl.Buffer(
    cl_ctx,
    cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR,
    hostbuf=plaintext_length)

cl_hash_output = cl.Buffer(
    cl_ctx,
    cl.mem_flags.WRITE_ONLY,
    hash_output.nbytes)

# execute the program
cl_program.get_single_hash(
    cl_queue, (1,), None,
    cl_plaintext_bytes,
    cl_plaintext_length,
    cl_hash_output)

# get the results
cl.enqueue_copy(cl_queue, hash_output, cl_hash_output)

# print the results
print("Plaintext: %s" % plaintext)
print("CL result      : %s" % hexlify(hash_output))
print("Correct result : %s" % hexlify(hashlib.sha256(plaintext.encode('utf-8')).digest()))
print("CL output      : %s" % hash_output)

assert hexlify(hash_output) == hexlify(hashlib.sha256(plaintext.encode('utf-8')).digest())


Plaintext: this is a description of the latest block395707976
CL result      : b'0000000053d18bbf3e8293e6a3bacda6e35ee7715e6353bf55a86cb9c3b4167b'
Correct result : b'0000000053d18bbf3e8293e6a3bacda6e35ee7715e6353bf55a86cb9c3b4167b'
CL output      : [         0 3213611347 3868426814 2798500515 1910988515 3209913182
 3110905941 2065085635]


### Random String Generation

In [176]:
# set up the variables to generate the random numbers
seed = np.random.randint(0, np.iinfo(np.uint32).max, dtype=np.uint32)
start = np.uint8(32)
end = np.uint8(126)
length = np.uint32(4)
result = np.zeros(length, dtype=np.uint8)

# allocate the memory for the variables on the device
cl_seed = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=seed)
cl_start = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=start)
cl_end = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=end)
cl_len = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=length)
cl_result = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, result.nbytes)

# execute the program
cl_program.get_random_numbers(
    cl_queue, (1,), None,
    cl_seed,
    cl_start,
    cl_end,
    cl_len,
    cl_result)

# get the results
cl.enqueue_copy(cl_queue, result, cl_result)

# print the results
print(f"Seed: {seed}")
print(f"Result: {result}")
print(f"Result String: {result.tobytes()}")

Seed: 1277229578
Result: [ 71  44  47 125]
Result String: b'G,/}'


In [177]:
# set up the variables to generate the random numbers
seed = np.random.randint(0, np.iinfo(np.uint32).max, dtype=np.uint32)
result = np.zeros(shape=16, dtype=np.uint8)
length = np.zeros(shape=(1,), dtype=np.uint8)

# allocate the memory for the variables on the device
cl_seed = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=seed)
cl_result = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, result.nbytes)
cl_len = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, length.nbytes)

# execute the program
cl_program.get_random_string(
    cl_queue, (1,), None,
    cl_seed,
    cl_result,
    cl_len)

# get the results
cl.enqueue_copy(cl_queue, length, cl_len)
cl.enqueue_copy(cl_queue, result, cl_result)

# print the results
print(f"Length: {length}")
print(f"Result: {result}")
print(f"Result String: {result.tobytes()}")

Length: [12]
Result: [ 32  33 104  84  64  53 103 107  79  92  62  84   0   0   0   0]
Result String: b' !hT@5gkO\\>T\x00\x00\x00\x00'


## Test get_single_hash_nonce

In [178]:
# set up the variables to generate the random numbers
plaintext = 'this is a description of the latest block'
plaintext_bytes = np.frombuffer(plaintext.encode('utf-8'), dtype=np.uint8)
plaintext_length = np.int32(len(plaintext_bytes))

seed = np.random.randint(0, np.iinfo(np.uint32).max, dtype=np.uint32)
nonce = np.zeros(shape=16, dtype=np.uint8)
nonce_len = np.zeros(shape=1, dtype=np.uint8)
hash = np.zeros(8, dtype=np.uint32)

# allocate the memory for the variables on the device
cl_seed = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=seed)
cl_plaintext_bytes = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=plaintext_bytes)
cl_plaintext_length = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=plaintext_length)
cl_nonce = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, nonce.nbytes)
cl_nonce_len = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, nonce_len.nbytes)
cl_hash_output = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, hash_output.nbytes)

# execute the program
cl_program.get_single_hash_nonce(
    cl_queue, (1,), None,
    cl_seed,
    cl_plaintext_bytes,
    cl_plaintext_length,
    cl_nonce,
    cl_nonce_len,
    cl_hash_output)

# get the results
cl.enqueue_copy(cl_queue, nonce, cl_nonce)
cl.enqueue_copy(cl_queue, nonce_len, cl_nonce_len)
cl.enqueue_copy(cl_queue, hash_output, cl_hash_output)

nonce_str = nonce[:nonce_len[0]].tobytes().decode('UTF-8')

# print the results
print("Seed: %s" % seed)
print("Plaintext: %s" % plaintext)
print("Nonce: %s" % nonce)
print("Nonce Length: %s" % nonce_len[0])
print("Nonce String: %s" % nonce_str)
print("Hash Output: %s" % hash_output)
print("Hash Output string: %s" % hexlify(hash_output))
#print("No Nonce          : %s" % hexlify(hashlib.sha256((plaintext).encode('utf-8')).digest()))
print("Correct result    : %s" % hexlify(hashlib.sha256((plaintext+nonce_str).encode('utf-8')).digest()))

assert hexlify(hash_output) == hexlify(hashlib.sha256((plaintext+nonce_str).encode('utf-8')).digest())


Seed: 1484029365
Plaintext: this is a description of the latest block
Nonce: [38 69  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
Nonce Length: 2
Nonce String: &E
Hash Output: [2290695762  422520507 3408130140 1217032290 3932659453 1858261943
 3702261654  358009833]
Hash Output string: b'523e8988bb262f195cf023cb62708a48fd9e67eab7d3c26e9607acdce9cb5615'
Correct result    : b'523e8988bb262f195cf023cb62708a48fd9e67eab7d3c26e9607acdce9cb5615'
